In [23]:
import torch

# Local GPU Test

print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

2.5.1
True
NVIDIA GeForce RTX 3090


In [24]:
import objaverse

uids = objaverse.load_uids() # each object has a unique ID (uid), then load it
print("all of objects length :", len(uids))

lvis_annotations = objaverse.load_lvis_annotations()

num_of_objects = 10
filtered_lvis_annotations = {key: value for key, value in lvis_annotations.items() if len(value) >= num_of_objects}
print(f"the number of classes having items more than {num_of_objects} : {len(filtered_lvis_annotations)}")

all of objects length : 798759
the number of classes having items more than 10 : 1061


In [25]:
my_datasets = {}

for key, value in filtered_lvis_annotations.items():
    sampled_uids = value[:num_of_objects]
    my_datasets[key] = sampled_uids
    print(f"{key} has {len(sampled_uids)} items : {sampled_uids}")
print(f"This dataset consists of {len(my_datasets)} classes.")

Bible has 10 items : ['ee7660359dda4c90990cb7440211b873', '3e5393bd35e34531a576e22d45611060', 'f5bf29193be44caaad2df1db3db44871', 'adde1e2669d14286843bc9f1cff9ae45', 'f0ed46d723dd441bb592d715db241097', '9d3007d40ca84bb7baec1f1a04e0f08d', '797e7c7f61104b6785c3bd0fdb99c74d', '149d47c6c27648a6af26d6c912c03826', '8257772b0e2f408ba269264855dfea00', 'e7b30aa0ad754744bb5344bc86bf794a']
CD_player has 10 items : ['b4af2a721b1645ddb77168d5e8fe4882', '830b1864b50e4f7e94c7f5cee5b64b43', '639a89472cfa4794b848175bbf652136', '00a66884dacb498b85701acd1cc77c5e', 'dbf0dbcf4229429088b0a984e8f05666', '6914d1e81eeb4377b61af0e5321ede5b', '73e76edb90914783b0fd732ea53b2e46', '9749f5ba1221476993376a3cb8fee1a5', 'f6c8386e76e54918b551ebdd276479dd', 'a4bc4accb20b43ffa728a7a85b7dfed7']
Christmas_tree has 10 items : ['e283aa7835664a74a4ad39afb26d2ef3', 'ae0c3996ee0345aaa8bbb717db7ffb25', 'fe2b4a2708334ff880f62a947cfd3d50', '2da008c410be42d188a0c95aa1bca05e', '16fbadc669454c26aa0927445b3c6ffd', '0b1b92d6d5584284b59a

In [26]:
# once you download the datasets, you don't need to run it again.

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

count = 1
for key, value in my_datasets.items():
    print(f"\nDownloading {key}... ({count} / {len(my_datasets)})")
    objaverse.load_objects(value, 1)
    print(f"Downloading {key} done.\n")
    count += 1

print("All downloading are done.")


Downloaded 1 / 10 objects
Downloaded 2 / 10 objects
Downloaded 3 / 10 objects
Downloaded 4 / 10 objects
Downloaded 5 / 10 objects
Downloaded 6 / 10 objects
Downloaded 7 / 10 objects
Downloaded 8 / 10 objects
Downloaded 9 / 10 objects
Downloaded 10 / 10 objects


Downloaded 1 / 10 objects
Downloaded 2 / 10 objects
Downloaded 3 / 10 objects
Downloaded 4 / 10 objects
Downloaded 5 / 10 objects
Downloaded 6 / 10 objects
Downloaded 7 / 10 objects
Downloaded 8 / 10 objects
Downloaded 9 / 10 objects
Downloaded 10 / 10 objects


Downloaded 1 / 10 objects
Downloaded 2 / 10 objects
Downloaded 3 / 10 objects
Downloaded 4 / 10 objects
Downloaded 5 / 10 objects
Downloaded 6 / 10 objects
Downloaded 7 / 10 objects
Downloaded 8 / 10 objects
Downloaded 9 / 10 objects
Downloaded 10 / 10 objects


Downloaded 1 / 10 objects
Downloaded 2 / 10 objects
Downloaded 3 / 10 objects
Downloaded 4 / 10 objects
Downloaded 5 / 10 objects
Downloaded 6 / 10 objects
Downloaded 7 / 10 objects
Downloaded 8 / 10 objects
Do

In [13]:
import json
import trimesh
import gzip
import pyrender
import numpy as np
import os
from PIL import Image

class Renderer:
    def __init__(self, path_file, output_path):
        self.my_datasets = my_datasets # key : a class, value : a list of uids
        self.base_path = os.path.join("D:\Studies\objaverse", "hf-objaverse-v1")
        with gzip.open(path_file, 'rt') as f:
            self.object_paths = json.load(f)
        self.output_path = output_path
        
    def load_mesh(self, uid):
        # load 3D .glb mesh from its UID
        mesh_path = os.path.join(self.base_path, self.object_paths[uid])
        if not mesh_path or not os.path.exists(mesh_path):
            print(f"Model file not found for UID: {uid}")
            return None
        
        # normalize the mesh to fit inside a 2*2*2 cube centered at (0,0,0)
        try:
            mesh = trimesh.load(mesh_path, force='mesh')
            centroid = mesh.bounds.mean(axis=0)        
            mesh.apply_translation(-centroid)
            
            max_extent = mesh.extents.max()
            scale_factor = 2.0 / max_extent
            
            mesh.apply_scale(scale_factor)
            
        except Exception as e:
            print(f"Error loading mesh: {e}")
            print(f"This mesh is not available (UID: {uid})")
            return None
        
        return mesh
    
    def render_mesh(self, uid, mesh, camera_pos=[[2.5, 1.0, 2.5], [-2.5, 1.0, 2.5], [-2.5, 1.0, -2.5], [2.5, 1.0, -2.5]], 
                    image_size=(512, 512)):
        
        try:
            scene = pyrender.Scene()
            mesh = pyrender.Mesh.from_trimesh(mesh)
            scene.add(mesh)
            
            camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0, name="camera")
            light = pyrender.DirectionalLight(color=np.ones(3), intensity=3.0, name="light")
            
        except Exception as e:
            print(f"This mesh is not available (UID: {uid})")
            return None    
        
        images = []
        
        for c_pos in camera_pos:
            # Set camera pose
            # Calculate camera pose to look at the origin
            eye = np.array(c_pos)
            target = np.array([0.0, 0.0, 0.0])  # Look at the origin
            up = np.array([0.0, 1.0, 0.0])  # Define the up direction

            # Compute the camera view matrix
            z_axis = (eye - target) / np.linalg.norm(eye - target) # Forward
            x_axis = np.cross(up, z_axis) / np.linalg.norm(np.cross(up, z_axis)) # Right
            y_axis = np.cross(z_axis, x_axis) # Up

            camera_pose = np.eye(4)
            camera_pose[:3, :3] = np.vstack([x_axis, y_axis, z_axis]).T
            camera_pose[:3, 3] = eye
            

            # Render the scene
            try:
                camera_node = scene.add(camera, pose=camera_pose)
                light_node = scene.add(light, pose=camera_pose)
                r = pyrender.OffscreenRenderer(*image_size)
                color, _ = r.render(scene)
                images.append(Image.fromarray(color))
                
                scene.remove_node(camera_node)
                scene.remove_node(light_node)
            except Exception as e:
                print(f"This mesh is not available (UID: {uid})")
                return None
                
        return images
    
    def process_dataset(self):
        cls_count = 1
        for cls, uids in self.my_datasets.items():
            print(f"\nProcessing {cls}... {cls_count} / {len(my_datasets)}")
            
            class_dir = os.path.join(self.output_path, cls)
            os.makedirs(class_dir, exist_ok=True)
            
            for uid in uids:
                img_exists = True
                for i in range(4):
                    img_path = os.path.join(class_dir, f"{uid}_{i}.png") # class path + img name
                    if os.path.exists(img_path):
                        print(f"The file {img_path} already exists.")
                    else:
                        img_exists = False
                        break
                
                if img_exists is False:
                    mesh = self.load_mesh(uid)
                    if mesh is None:
                        continue
                    images = self.render_mesh(uid, mesh)
                    if images is None:
                        continue
                    
                    for i, img in enumerate(images):
                        img_path = os.path.join(class_dir, f"{uid}_{i}.png")
                        try:
                            img.save(img_path)
                            print(f"Saved {uid}_{i}.png")
                        except Exception as e:
                            print(f"Error saving image: {e}")
            
            cls_count += 1
        
        print("Rendering completed!")

In [14]:
renderer = Renderer("D:\Studies\objaverse\hf-objaverse-v1\object-paths.json.gz", "D:\Studies\output")
renderer.process_dataset()


Processing Bible... 1 / 1061
The file D:\Studies\output\Bible\ee7660359dda4c90990cb7440211b873_0.png already exists.
The file D:\Studies\output\Bible\ee7660359dda4c90990cb7440211b873_1.png already exists.
The file D:\Studies\output\Bible\ee7660359dda4c90990cb7440211b873_2.png already exists.
The file D:\Studies\output\Bible\ee7660359dda4c90990cb7440211b873_3.png already exists.
The file D:\Studies\output\Bible\3e5393bd35e34531a576e22d45611060_0.png already exists.
The file D:\Studies\output\Bible\3e5393bd35e34531a576e22d45611060_1.png already exists.
The file D:\Studies\output\Bible\3e5393bd35e34531a576e22d45611060_2.png already exists.
The file D:\Studies\output\Bible\3e5393bd35e34531a576e22d45611060_3.png already exists.
The file D:\Studies\output\Bible\f5bf29193be44caaad2df1db3db44871_0.png already exists.
The file D:\Studies\output\Bible\f5bf29193be44caaad2df1db3db44871_1.png already exists.
The file D:\Studies\output\Bible\f5bf29193be44caaad2df1db3db44871_2.png already exists.
Th